<a href="https://colab.research.google.com/github/nithingovindugari/IntrotoQuantum_Project/blob/main/Project_Phase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from math import sqrt

In [ ]:
def getTensor(matrices):
    product = matrices[0]
    for matrix in matrices[1:]:
        product = np.kron(product,matrix)  ## np.kron stands for Kronecker product, the official name for tensor product
    return product

In [ ]:
def U(n, f_map):
    """Generate an oracle matrix based on the given function mapping."""
    # INSPIRED BY https://github.com/meownoid/quantum-python/blob/master/quantum.py
    
    num_qubits = n + 1
    U = np.zeros((2**num_qubits, 2**num_qubits)) # Start with a matrix of zeroes.
    
    # Quantum state looks like IN-IN-...-IN-ANCILLA
    for input_state in range(2**num_qubits): # For each possible input
        input_string = input_state >> 1 # remove ANCILLA
        output_qubit = (input_state & 1) ^ (f_map[input_string]) # remove IN, XOR with f(IN)
        output_state = (input_string << 1) + output_qubit # the full state, with new OUT
        U[input_state, output_state] = 1 # set that part of U to 1

    return U

In [ ]:
def measure(n, state):
    measurement = np.zeros(2**n)  # Initialize measurement result for n qubits in the first register
    for index, value in enumerate(state):
        measurement[index >> 1] += value * value  ## As the ancilla qubit is discarded, probabilities of the same kind, ie 100 and 101 will be combined

    # Last step: Determine the type of function f
    # f is constant if the probability of measuring |0> is positive
    if (abs(measurement[0]) > 1e-10): 
        print("The function is constant.")
    else:
        print("The function is balanced.")

In [ ]:
def Deutsch_Jozsa(n, f_map):
    num_qubits = n + 1  # Plus one qubit and the second register, can be called as ancilla qubit
    state_0 = np.array([[1],[0]])  # Standard state |0> as a column vector
    I_gate = np.array([[1,0], [0,1]])  # Identity gate
    X_gate = np.array([[0,1], [1,0]])  # NOT gate
    H_gate = np.array([[1,1], [1,-1]])/sqrt(2)  # Hadamard gate
    
    ancilla = np.dot(X_gate, state_0)  # Create state |1> assigned to the ancilla
    
    # Create the a Hadamard transformation for all qubits and the state |ψ_0> 
    listStates = []
    listGates_H = []
    for i in range(n):
        listStates.append(state_0)
        listGates_H.append(H_gate)
    listStates.append(ancilla)
    listGates_H.append(H_gate)
    psi_0 = getTensor(listStates)
    composite_H = getTensor(listGates_H)
    
    # |ψ_1> is the dot product of the Hadamard transformation and |ψ_0>  
    psi_1 = np.dot(composite_H, psi_0)

    # Apply the oracle to |ψ_1>
    psi_2 = np.dot(U(n, f_map), psi_1)

    # H on all again
    psi_3 = np.dot(composite_H, psi_2)

    measure(n, psi_3)

In [ ]:
def main():
    n = [2,3,3]  # Input the number of qubits
    f_map = [[0,0,1,1],
             [1,1,1,1,1,1,1,1],
             [1,0,0,1,1,0,1,0]]  # Input the mapping functions
    for index, value in enumerate(n):
        Deutsch_Jozsa(n[index], f_map[index])  # Algorithm executed here

In [ ]:
main()

The function is balanced.
The function is constant.
The function is balanced.


In [ ]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 12.7 MB/s 
     |████████████████████████████████| 19.2 MB 1.6 MB/s 
     |████████████████████████████████| 240 kB 45.7 MB/s 
     |████████████████████████████████| 106 kB 68.4 MB/s 
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 49 kB 4.3 MB/s 
     |████████████████████████████████| 929 kB 56.1 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 50 kB 3.6 MB/s 
     |████████████████████████████████| 4.0 MB 29.7 MB/s 
     |████████████████████████████████| 1.9 MB 60.1 MB/s 
     |████████████████████████████████| 112 kB 58.5 MB/s 
  Created wheel for qiskit: filename=qiskit-0.39.3-py3-none-any.whl size=12262 sha256=41e186b1b52dd055437c1c2fc3736111bb7a0d79f8b98b4ce77887e5d9839226
  Stored in directory: /root/.cache/pip/wheels/46/24/40/7b6f

In [ ]:
'''
    Deutsch-Jozsa Algorithm
    Consider a function f(x) that takes as input n-bit strings x and returns 0 or 1. Suppose we are
    promised that f(x) is either a constant function that takes the same value c in {0,1} on all
    inputs x, or a balanced function that takes each value 0 and 1 on exactly half of the inputs. 
    The goal is to decide whether f is constant or balanced by making as few function evaluations 
    as possible. Classically, it requires 2^{n-1}+1 function evaluations in the worst case. Using 
    the Deutsch-Jozsa algorithm, the question can be answered with just one function evaluation.
    
    Deutsch's algorithm is the simpler case of Deutsch-Jozsa Algorithm which has a function f(x) 
    which takes 1-bit as input.
    Source: https://github.com/Qiskit/ibmqx-user-guides/blob/master/rst/full-user-guide/004-Quantum_Algorithms/080-Deutsch-Jozsa_Algorithm.rst
'''
from qiskit import IBMQ, BasicAer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.monitor import job_monitor

qr = QuantumRegister(2)  # Initialize two qubits
cr = ClassicalRegister(2)  # Initialize two bits for record measurements
circuit = QuantumCircuit(qr, cr)

circuit.x(qr[1])  # initialize the ancilla qubit in the |1> state

circuit.barrier()

# First step of quantum algorithms - Prepare the superposition
# For superposition, we apply the Hadamard gate on both qubits
circuit.h(qr[0])
circuit.h(qr[1])

circuit.barrier()

# Oracle function
circuit.cx(qr[0], qr[1])

circuit.barrier()

# Apply Hadamard gates after querying oracle function
circuit.h(qr[0])
circuit.h(qr[1])

circuit.barrier()

# Measure qubit
circuit.measure(qr[0], cr[0])

# Run our circuit with local simulator
backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(circuit, backend=backend, shots=shots).result()
answer = results.get_counts()
print("Simulator result")
for c1c0 in answer:
    print(f'c0 = {c1c0[1]} ({answer[c1c0]} shots)')
# C0 observed as 1 in 1024 shots
# It indicates f(0) != f(1)

# # Run our circuit with real devices
# IBMQ.load_account()
# IBMQ.backends()
# backend_lb = least_busy(IBMQ.backends(simulator=False))
# backend = backend_lb
# shots = 1024
# job_exp = execute(circuit, backend=backend, shots=shots)
# job_monitor(job_exp, interval=2)
# results = job_exp.result()
# answer = results.get_counts(circuit)
# print("Real Device Result")
# for c1c0 in answer:
#     print(f'c0 = {c1c0[1]} ({answer[c1c0]} shots)')
# As we can see in results, most of the results for C0 is 1
# It indicates f(0) != f(1)
# The results with C0 = 0 occur due to errors in the quantum computation.

Simulator result
c0 = 1 (1024 shots)


In [ ]:
!pip install cirq

In [ ]:
# pylint: disable=wrong-or-nonexistent-copyright-notice
"""Demonstrates Deutsch's algorithm.
Deutsch's algorithm is one of the simplest demonstrations of quantum parallelism
and interference. It takes a black-box oracle implementing a Boolean function
f(x), and determines whether f(0) and f(1) have the same parity using just one
query.  This version of Deutsch's algorithm is a simplified and improved version
from Nielsen and Chuang's textbook.
=== REFERENCE ===
https://en.wikipedia.org/wiki/Deutsch–Jozsa_algorithm
Deutsch, David. "Quantum theory, the Church-Turing Principle and the universal
quantum computer." Proc. R. Soc. Lond. A, 400:97, 1985.
=== EXAMPLE OUTPUT ===
Secret function:
f(x) = <0, 1>
Circuit:
0: ───────H───@───H───M('result')───
              │
1: ───X───H───X─────────────────────
Result f(0)⊕f(1):
result=1
"""

import random

import cirq
from cirq import H, X, CNOT, measure


def main():
    # Choose qubits to use.
    q0, q1 = cirq.LineQubit.range(2)

    # Pick a secret 2-bit function and create a circuit to query the oracle.
    secret_function = [random.randint(0, 1) for _ in range(2)]
    oracle = make_oracle(q0, q1, secret_function)
    print(f"Secret function:\nf(x) = <{', '.join(str(e) for e in secret_function)}>")

    # Embed the oracle into a quantum circuit querying it exactly once.
    circuit = make_deutsch_circuit(q0, q1, oracle)
    print('Circuit:')
    print(circuit)

    # Simulate the circuit.
    # simulator = cirq.Simulator()
    # result = simulator.run(circuit)
    print('Result of f(0)⊕f(1):')
    # print(result)


def make_oracle(q0, q1, secret_function):
    """Gates implementing the secret function f(x)."""

    # coverage: ignore
    if secret_function[0]:
        yield [CNOT(q0, q1), X(q1)]

    if secret_function[1]:
        yield CNOT(q0, q1)


def make_deutsch_circuit(q0, q1, oracle):
    # c = cirq.Circuit()

    # Initialize qubits.
    # c.append([X(q1), H(q1), H(q0)])
    listStates = []
    listStates.append([X(q1), H(q1), H(q0)])
    # Query oracle.
    # c.append(oracle)
    listStates.append(oracle)

    # Measure in X basis.
    listStates.append([H(q0)])
    return listStates


if __name__ == '__main__':
    main()

Secret function:
f(x) = <0, 1>
Circuit:
[[cirq.X(cirq.LineQubit(1)), cirq.H(cirq.LineQubit(1)), cirq.H(cirq.LineQubit(0))], <generator object make_oracle at 0x7f3abd329970>, [cirq.H(cirq.LineQubit(0))]]
Result of f(0)⊕f(1):


In [ ]:
!pip install cirq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# pylint: disable=wrong-or-nonexistent-copyright-notice
"""Demonstrates Deutsch's algorithm.
Deutsch's algorithm is one of the simplest demonstrations of quantum parallelism
and interference. It takes a black-box oracle implementing a Boolean function
f(x), and determines whether f(0) and f(1) have the same parity using just one
query.  This version of Deutsch's algorithm is a simplified and improved version
from Nielsen and Chuang's textbook.
=== REFERENCE ===
https://en.wikipedia.org/wiki/Deutsch–Jozsa_algorithm
Deutsch, David. "Quantum theory, the Church-Turing Principle and the universal
quantum computer." Proc. R. Soc. Lond. A, 400:97, 1985.
=== EXAMPLE OUTPUT ===
Secret function:
f(x) = <0, 1>
Circuit:
0: ───────H───@───H───M('result')───
              │
1: ───X───H───X─────────────────────
Result f(0)⊕f(1):
result=1
"""

import random

import cirq
from cirq import H, X, CNOT, measure


def main():
    # Choose qubits to use.
    q0, q1 = cirq.LineQubit.range(2)

    # Pick a secret 2-bit function and create a circuit to query the oracle.
    secret_function = [random.randint(0, 1) for _ in range(2)]
    oracle = make_oracle(q0, q1, secret_function)
    print(f"Secret function:\nf(x) = <{', '.join(str(e) for e in secret_function)}>")

    # Embed the oracle into a quantum circuit querying it exactly once.
    circuit = make_deutsch_circuit(q0, q1, oracle)
    print('Circuit:')
    print(circuit)

    # Simulate the circuit.
    simulator = cirq.Simulator()
    result = simulator.run(circuit)
    print('Result of f(0)⊕f(1):')
    print(result)


def make_oracle(q0, q1, secret_function):
    """Gates implementing the secret function f(x)."""

    # coverage: ignore
    if secret_function[0]:
        yield [CNOT(q0, q1), X(q1)]

    if secret_function[1]:
        yield CNOT(q0, q1)


def make_deutsch_circuit(q0, q1, oracle):
    c = cirq.Circuit()

    # Initialize qubits.
    c.append([X(q1), H(q1), H(q0)])

    # Query oracle.
    c.append(oracle)

    # Measure in X basis.
    c.append([H(q0), measure(q0, key='result')])
    return c


if __name__ == '__main__':
    main()

Secret function:
f(x) = <0, 0>
Circuit:
0: ───H───H───M('result')───

1: ───X───H─────────────────
Result of f(0)⊕f(1):
result=0
